<a href="https://colab.research.google.com/github/diazdela/Semantic-Segmentation-of-Breast-Cancer-Tissue/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project - Breast Cancer Semantic Segmentation

## Dataset https://github.com/CancerDataScience/CrowdsourcingDataset-Amgadetal2019.git

In [1]:
# Clone the repository
!git clone https://github.com/CancerDataScience/CrowdsourcingDataset-Amgadetal2019.git
%cd CrowdsourcingDataset-Amgadetal2019

# Install the required packages
!pip install girder_client pillow numpy scikit-image imageio

!python download_crowdsource_dataset.py


Cloning into 'CrowdsourcingDataset-Amgadetal2019'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 230 (delta 47), reused 19 (delta 4), pack-reused 139
Receiving objects: 100% (230/230), 74.54 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (107/107), done.
/content/CrowdsourcingDataset-Amgadetal2019
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.3 MB/s eta 0:00:00
  Created wheel for girder_client: filename=girder_client-3.1.17-py3-none-any.whl size=21179 sha256=7304d4d4ee61f7de5fcbdc145b0178d1da490706638fe12592b2b2abf01074b8
  Stored in directory: /root/.cache/pip/wheels/1c/88/79/eb68de788a8c7a6cf3d101905b31a05dc69aa9ac212f5db922
Successfully built girde

In [2]:
# Identifying Directories
import os
os.getcwd()

# Parent Directory
parent_dir = "/content/CrowdsourcingDataset-Amgadetal2019"

# Sub-directories
sub_dirs = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]
print(sub_dirs)

['images', 'masks', 'wsis', 'annotations', 'logs', '.git', '__pycache__', 'meta']


In [4]:
from PIL import Image
import matplotlib.pyplot as plt

# Visualize Images
img_sub_dir = "images"

# Path to the sub-directory
img_sub_path = os.path.join(parent_dir, img_sub_dir)

# List files in sub-directory
img_sub_files = os.listdir(img_sub_path)

# Print the list of files in the sub-directory
print(img_sub_files)

# List files in the sub-directory
img_file_1 = img_sub_files[1]
file_path = os.path.join(img_sub_path, img_file_1)

# Visualize
image = Image.open(file_path)
height=480
img_1 = image.resize((int(float(image.size[0])*height/float(image.size[1])),height))

plt.rcParams["figure.figsize"] = (5,5)
plt.imshow(img_1); plt.show()

['TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0.2500.png']


IndexError: ignored

## Model https://github.com/IanTaehoonYoo/semantic-segmentation-pytorch

In [5]:
!pip install seg-torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.9 MB/s eta 0:00:00


In [9]:
from torchvision import transforms

from segmentation.data_loader.segmentation_dataset import SegmentationDataset
from segmentation.data_loader.transform import Rescale, ToTensor
from segmentation.trainer import Trainer
from segmentation.predict import *
from segmentation.models import all_models
from util.logger import Logger

train_images = r'/content/CrowdsourcingDataset-Amgadetal2019/images'
train_labled = r'/content/CrowdsourcingDataset-Amgadetal2019/masks'

if __name__ == '__main__':
    model_name = "fcn8_vgg16"
    device = 'cuda'
    batch_size = 4
    n_classes = 34
    num_epochs = 10
    image_axis_minimum_size = 200
    pretrained = True
    fixed_feature = False
    image_target_size = (200, 200)

    logger = Logger(model_name=model_name, data_name='example')

    ### Loader
    compose = transforms.Compose([
        transforms.Resize(image_target_size),  # Resize images to a consistent size
        ToTensor()
         ])

    train_datasets = SegmentationDataset(train_images, train_labled, n_classes, compose)
    train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

#    test_datasets = SegmentationDataset(test_images, test_labeled, n_classes, compose)
#    test_loader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

    ### Model
    model = all_models.model_from_name[model_name](n_classes, batch_size,
                                                   pretrained=pretrained,
                                                   fixed_feature=fixed_feature)
    model.to(device)

    ###Load model
    ###please check the foloder: (.segmentation/test/runs/models)
    #logger.load_model(model, 'epoch_15')

    ### Optimizers
    if pretrained and fixed_feature: #fine tunning
        params_to_update = model.parameters()
        print("Params to learn:")
        params_to_update = []
        for name, param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t", name)
        optimizer = torch.optim.Adadelta(params_to_update)
    else:
        optimizer = torch.optim.Adadelta(model.parameters())


    ### Train
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    trainer = Trainer(model, optimizer, logger, num_epochs, train_loader)
    trainer.train()


    #### Writing the predict result.
    predict(model, r'/content/CrowdsourcingDataset-Amgadetal2019/images',
             r'/content/CrowdsourcingDataset-Amgadetal2019/masks')

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.15it/s]


Dataset verified! 


Train:   0%|                                             | 0/10 [00:00<?, ?it/s]
Train epoch=0: 0it [00:00, ?it/s]
Train:  10%|███▋                                 | 1/10 [00:00<00:01,  7.25it/s]
Train epoch=1: 0it [00:00, ?it/s]
                                 
Train epoch=2: 0it [00:00, ?it/s]
                                 
Train epoch=3: 0it [00:00, ?it/s]
                                 
Train epoch=4: 0it [00:00, ?it/s]
                                 
Train epoch=5: 0it [00:00, ?it/s]
                                 
Train epoch=6: 0it [00:00, ?it/s]
                                 
Train epoch=7: 0it [00:00, ?it/s]
                                 
Train epoch=8: 0it [00:00, ?it/s]
                                 
Train epoch=9: 0it [00:00, ?it/s]
Train: 100%|████████████████████████████████████| 10/10 [00:00<00:00, 45.78it/s]


AttributeError: ignored